In [1]:
import requests
import json
import os
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
class Weather_API:
    
    def __init__ (self, keyword):
        self.keyword = keyword
        
    def json_print(self, obj):
        # create a formatted string of the Python JSON object
        with open('api_data.txt', 'w') as json_file:
            json.dump(obj, json_file)
        text = json.dumps(obj, sort_keys=True, indent=4)
        print(text)
        
    def create_dataframe(self, obj):
        
        # creating a dataframe from nested JSON objects
        FIELDS = ["source.id", "source.name", "author", "title", "description", "url", "urlToImage", "publishedAt", "content"]
        df = pd.json_normalize(obj['articles'])
        final_df = df[FIELDS]
        #final_df.set_index('source.id', inplace = True)
        display(final_df.head())

    
    def news_api(self):
        
        # Use the news-api to obtain articles published from
        url = ('https://newsapi.org/v2/everything?'
       'q={keyword}&'
       'apiKey=4e70cabb80884db08524a28ac33cdc1d'.format(keyword = self.keyword))
        
        
        response = requests.get(url)
        if (response.status_code == 200):
            print('API call successful!')
            json_response = response.json()
            if(len(json_response['articles']) == 0):
                print('No News Articles Found')
            else:
                
                # Print a String in Json Format
                self.json_print(json_response)
                
                
                
                # Create a pandas DataFrame
                self.create_dataframe(json_response)
                    
        else:
            print('Status code: ', response.status_code)

In [3]:
class Web_Scraping:
    
    def __init__(self, location):
        self.location = location
        
    def selenium_webdriver(self):
        
        # Start the Driver
        driver = webdriver.Chrome(executable_path = r"C:\Users\Public\Desktop\Google Chrome.lnk")
        
        # Hit the url of NASA Earth Data website and wait for 15 seconds.
        url = ('https://earthdata.nasa.gov/search?q={location}'.format(location = self.location))
        driver.get(url)
        time.sleep(15)
        
        # Driver scrolls down 25 times to load the table.
        for i in range(0,30):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(10)
            
        # Fetch the webpage and store in a variable.
        webpage = driver.page_source
        
        # Parse the page using BeautifulSoup
        HTMLPage = BeautifulSoup(webpage, 'html.parser')
        
        titles = []
        description = []
        links = []

        for lists in HTMLPage.find_all(class_ = 'result'):
            if (lists.span.text != '' and len(lists.find_all('p')) != 0):
                titles.append(lists.span.text)
                description.append(lists.find('p', class_ = '').text)
                links.append(lists.find('p', class_ = 'search-link').text)
        
        # Create a DataFrame
        df = pd.DataFrame(list(zip(titles, description, links)),
               columns =['title', 'description', 'link'])
        
        display(df)
        
        # Store to csv file
        df.to_csv('ws.csv', sep=',', index=False,header=True)
        
        print('Web Scraping Successful!')

In [ ]:
keyword = input('Enter Keyword to be searched: ').lower()
w_api = Weather_API(keyword)
w_api.news_api()

location = input('Enter Location: ').lower()
ws = Web_Scraping('India')
ws.selenium_webdriver()

Enter Keyword to be searched:  chennai


API call successful!
{
    "articles": [
        {
            "author": "Ryan Hogg",
            "content": "More employers are pushing for workers to return to the office in a hybrid role or full-time.Craig F. Walker/The Boston Globe via Getty Images\r\n<ul>\n<li>A new study looks at the productivity of remot\u2026 [+2072 chars]",
            "description": "A new study found that a group of fully remote data entry workers in India were 18% less productive than those that worked in the office.",
            "publishedAt": "2023-08-06T11:07:35Z",
            "source": {
                "id": "business-insider",
                "name": "Business Insider"
            },
            "title": "New research finds that fully remote workers are less productive than their in-office counterparts",
            "url": "https://www.businessinsider.com/wfh-workers-less-efficient-than-people-in-office-new-research-2023-8",
            "urlToImage": "https://i.insider.com/64ac30ec6d7e02001af8a88d?wi

,source.id,source.name,author,title,description,url,urlToImage,publishedAt,content
0,business-insider,Business Insider,Ryan Hogg,New research finds that fully remote workers a...,A new study found that a group of fully remote...,https://www.businessinsider.com/wfh-workers-le...,https://i.insider.com/64ac30ec6d7e02001af8a88d...,2023-08-06T11:07:35Z,More employers are pushing for workers to retu...
1,None,Marginalrevolution.com,Tyler Cowen,Growth models and new goods,Growth models typically assume an inaccurate e...,https://marginalrevolution.com/marginalrevolut...,https://marginalrevolution.com/wp-content/uplo...,2023-08-06T06:19:32Z,Growth models typically assume an inaccurate e...
2,None,Marginalrevolution.com,Tyler Cowen,Murugan Idli,"E. 149, Murugan Idli Shop, 1, 6th Avenue, opp....",https://marginalrevolution.com/marginalrevolut...,https://marginalrevolution.com/wp-content/uplo...,2023-08-06T19:31:35Z,"E. 149, Murugan Idli Shop, 1, 6th Avenue, opp...."
3,None,Slickdeals.net,serra,"Roundtrip Flight (1-Stop): New York, New York ...",Delta Air Lines offers Roundtrip Flights from ...,https://slickdeals.net/f/16856732-roundtrip-fl...,https://static.slickdealscdn.com/attachment/1/...,2023-08-15T16:54:18Z,Delta Air Lines offers Roundtrip Flights from ...
4,None,The Indian Express,Express Web Desk,Tamil Nadu Minister K N Nehru initiates work f...,"The minister alongside DMK MP Dayanidhi Maran,...",https://indianexpress.com/article/cities/chenn...,https://images.indianexpress.com/2023/09/chenn...,2023-09-02T06:12:00Z,Tamil Nadu Minister for Municipal Administrati...
